In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import nltk
import re
import string
from nltk.corpus import stopwords
from transformers import AutoTokenizer
from bs4 import BeautifulSoup
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
df = pd.read_json('combined_dataset.jsonl', lines=True)
data = df[['paragraph', 'labels']]

# Display dataset info
print(data.info())
print(data.labels.value_counts())
print(data.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   paragraph  8000 non-null   object
 1   labels     8000 non-null   object
dtypes: object(2)
memory usage: 125.1+ KB
None
labels
[no_technique]                                                                                                   2966
[Loaded_Language]                                                                                                1513
[Name_Calling-Labeling, Loaded_Language]                                                                          406
[Loaded_Language, Exaggeration-Minimisation]                                                                      322
[Obfuscation-Vagueness-Confusion, Loaded_Language]                                                                181
                                                                                              

In [ ]:
df = pd.read_json('combined_dataset.jsonl', lines=True)
data = df[['paragraph', 'labels']]
data

,paragraph,labels,stop_words,label_ids
0,ندوه سلب اسكندرون الي سلب فلسطين مدرج دار البع...,"[Flag_Waving, Loaded_Language, Exaggeration-Mi...",3,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, ..."
1,استقلال الجزاءر تزوجت جميله بوحيرد بالمحامي ال...,[Name_Calling-Labeling],7,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,تفصح الخاطر فحوي حمله وزير الخارجيه الكويتي اح...,[Loaded_Language],8,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
3,وبدورها كتبت نور الهجري فيسبوك احتفال بعيد الا...,"[Loaded_Language, Causal_Oversimplification]",6,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, ..."
4,وظل بوتفليقه سده الرءاسه الجزاءريه يظهر علني ا...,[Doubt],4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
7995,اجري رءيس الاركان العامه للقوات المسلحه الروسي...,[no_technique],1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7996,ومنعت جبهه تحرير شعب تيغراي الجمعه جنرالا عينت...,[no_technique],4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7997,تتميز هونغ كونغ بالعديد المعالم السياحيه والتر...,[no_technique],4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7998,وتابع بعدما تفرقت اراء الصحابه يتفقوا شء معين ...,[no_technique],5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
data['paragraph']

0       ندوة «من سلب اسكندرون إلى سلب فلسطين» على مدرج...
1       بعد استقلال الجزائر، تزوجت جميلة بوحيرد بالمحا...
2       وفي الوقت الذي لم تفصح فيه الخاطر عن فحوى ما ح...
3       وبدورها، كتبت نور الهجري، على فيسبوك " احتفال ...
4       وظل بوتفليقة في سدة الرئاسة الجزائرية نحو 20 ع...
                              ...                        
7995    أجرى رئيس الأركان العامة للقوات المسلحة الروسي...
7996    ومنعت جبهة تحرير شعب تيغراي يوم الجمعة الماضي ...
7997    تتميز هونغ كونغ بالعديد من المعالم السياحية وا...
7998    وتابع «بعدما تفرقت آراء الصحابة ولم يتفقوا على...
7999    وقالت الوزارة في بيان إن الإصابة الثانية المؤك...
Name: paragraph, Length: 8000, dtype: object

In [ ]:
arabic_stopwords = stopwords.words("arabic")
len(arabic_stopwords)
data['stop_words'] = data['paragraph'].apply(lambda x: len(set(x.split()) & set(arabic_stopwords)))
data.stop_words.value_counts()

<ipython-input-4-e5bead74d4c2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['stop_words'] = data['paragraph'].apply(lambda x: len(set(x.split()) & set(arabic_stopwords)))


stop_words
4     1119
5     1086
3     1079
6      940
7      792
2      670
8      601
9      394
10     333
1      315
11     207
12     148
0       98
13      91
14      54
15      31
16      20
17      11
18       6
20       3
19       1
24       1
Name: count, dtype: int64

In [ ]:
!wget https://raw.githubusercontent.com/alaa-a-a/multi-dialect-arabic-stop-words/main/Stop-words/stop_list_1177.txt
arabic_stop_words = []
with open ('./stop_list_1177.txt',encoding='utf-8') as f :
    for word in f.readlines() :
        arabic_stop_words.append(word.split("\n")[0])

--2024-07-27 19:48:49--  https://raw.githubusercontent.com/alaa-a-a/multi-dialect-arabic-stop-words/main/Stop-words/stop_list_1177.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11468 (11K) [text/plain]
Saving to: ‘stop_list_1177.txt.1’

stop_list_1177.txt. 100%[===================>]  11.20K  --.-KB/s    in 0s      

2024-07-27 19:48:49 (22.5 MB/s) - ‘stop_list_1177.txt.1’ saved [11468/11468]



In [ ]:
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

def remove_diacritics(text):
    arabic_diacritics = re.compile(r"""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    return re.sub(arabic_diacritics, '', text)

def remove_punctuations(text):
    arabic_punctuations = '''`÷×؛<>_«»()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–'''
    english_punctuations = string.punctuation
    punctuations_list = arabic_punctuations + english_punctuations
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1\1', text)  # keep 2 repeats

def remove_stop_words(text):
    word_list = nltk.tokenize.wordpunct_tokenize(text.lower())
    word_list = [w for w in word_list if not w in arabic_stop_words]
    return (" ".join(word_list)).strip()

def remove_non_arabic_letters(text):
    text = re.sub(r'([@A-Za-z0-9_]+)|#|http\S+', ' ', text)  # removes non-arabic letters
    return text

def clean_str(text):
    text = normalize_arabic(text)
    text = remove_non_arabic_letters(text)
    text = remove_punctuations(text)
    text = remove_diacritics(text)
    text = remove_repeating_char(text)
    text = remove_stop_words(text)

    soup = BeautifulSoup(text, 'lxml')
    text = soup.get_text()

    words = nltk.tokenize.wordpunct_tokenize(text)
    return (" ".join(words)).strip()

In [ ]:
data["paragraph"] = data["paragraph"].apply(lambda x: clean_str(x))

data.head()

<ipython-input-7-42941fd20a2d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["paragraph"] = data["paragraph"].apply(lambda x: clean_str(x))


,paragraph,labels,stop_words
0,ندوه سلب اسكندرون الي سلب فلسطين مدرج دار البع...,"[Flag_Waving, Loaded_Language, Exaggeration-Mi...",3
1,استقلال الجزاءر تزوجت جميله بوحيرد بالمحامي ال...,[Name_Calling-Labeling],7
2,تفصح الخاطر فحوي حمله وزير الخارجيه الكويتي اح...,[Loaded_Language],8
3,وبدورها كتبت نور الهجري فيسبوك احتفال بعيد الا...,"[Loaded_Language, Causal_Oversimplification]",6
4,وظل بوتفليقه سده الرءاسه الجزاءريه يظهر علني ا...,[Doubt],4


In [ ]:
mlb = MultiLabelBinarizer()
data['label_ids'] = mlb.fit_transform(data['labels']).tolist()
data.head()

<ipython-input-8-0c70af524f82>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['label_ids'] = mlb.fit_transform(data['labels']).tolist()


,paragraph,labels,stop_words,label_ids
0,ندوه سلب اسكندرون الي سلب فلسطين مدرج دار البع...,"[Flag_Waving, Loaded_Language, Exaggeration-Mi...",3,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, ..."
1,استقلال الجزاءر تزوجت جميله بوحيرد بالمحامي ال...,[Name_Calling-Labeling],7,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,تفصح الخاطر فحوي حمله وزير الخارجيه الكويتي اح...,[Loaded_Language],8,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
3,وبدورها كتبت نور الهجري فيسبوك احتفال بعيد الا...,"[Loaded_Language, Causal_Oversimplification]",6,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, ..."
4,وظل بوتفليقه سده الرءاسه الجزاءريه يظهر علني ا...,[Doubt],4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."


In [ ]:
data['label_ids']

0       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, ...
1       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...
3       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, ...
4       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...
                              ...                        
7995    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
7996    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
7997    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
7998    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
7999    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: label_ids, Length: 8000, dtype: object

In [ ]:
!pip install datasets # Install the datasets library using pip
!pip install --upgrade pyarrow
from datasets import Dataset, DatasetDict


In [ ]:
# Train/test split
train_df, test_df = train_test_split(data, test_size=0.2)

train_df, val_df = train_test_split(train_df, test_size=0.25)

train_df = train_df.drop(columns=['labels'])
val_df = val_df.drop(columns=['labels'])
test_df = test_df.drop(columns=['labels'])

train_df = train_df.drop(columns=['stop_words'])
val_df = val_df.drop(columns=['stop_words'])
test_df = test_df.drop(columns=['stop_words'])
# Convert to HuggingFace datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)
dataset = DatasetDict({"train": train_dataset,"val":val_dataset ,"test": test_dataset})
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['paragraph', 'label_ids', '__index_level_0__'],
        num_rows: 4800
    })
    val: Dataset({
        features: ['paragraph', 'label_ids', '__index_level_0__'],
        num_rows: 1600
    })
    test: Dataset({
        features: ['paragraph', 'label_ids', '__index_level_0__'],
        num_rows: 1600
    })
})


In [ ]:
model_name = "Bmalmotairy/arabert-fully-supervised-arabic-propaganda"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the pre-trained model without specifying the number of labels
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Modify the classification layer to match the number of labels
num_labels = len(data['label_ids'][0])
model.classifier = torch.nn.Linear(model.config.hidden_size, num_labels)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/751k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.77M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/879 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/541M [00:00<?, ?B/s]

In [ ]:
# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['paragraph'], padding="max_length", truncation=True)

# Tokenize the datasets
tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/4800 [00:00<?, ? examples/s]

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

In [ ]:
# Data collator
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Define compute_metrics function
from sklearn.metrics import f1_score, accuracy_score

def compute_metrics(p):
    preds = torch.sigmoid(torch.tensor(p.predictions)).round().numpy()
    labels = p.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}


# Define a custom Trainer class to use BCEWithLogitsLoss
from transformers import Trainer

class MultiLabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.BCEWithLogitsLoss()
        loss = loss_fct(logits, labels.float())
        return (loss, outputs) if return_outputs else loss


In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = MultiLabelTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Fine-tune the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.184500,0.143765,0.371250,0.419194,0.428840,0.409988
2,0.136100,0.136988,0.397500,0.469804,0.573484,0.454897
3,0.123000,0.138333,0.405625,0.480426,0.578274,0.468060


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=1800, training_loss=0.14243370903862848, metrics={'train_runtime': 1557.8443, 'train_samples_per_second': 9.244, 'train_steps_per_second': 1.155, 'total_flos': 3789547595366400.0, 'train_loss': 0.14243370903862848, 'epoch': 3.0})

In [ ]:
eval_results = trainer.evaluate()

print(f"Accuracy: {eval_results['eval_accuracy']}")
print(f"F1 Score: {eval_results['eval_f1']}")
print(f"Precision: {eval_results['eval_precision']}")
print(f"Recall: {eval_results['eval_recall']}")

Accuracy: 0.405625
F1 Score: 0.4804264164830517
Precision: 0.5782735720607938
Recall: 0.46806039488966317


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Evaluate the model on the test dataset
test_results = trainer.evaluate(tokenized_datasets["test"])

# Print test results
print(f"Test Accuracy: {test_results['eval_accuracy']:.4f}")
print(f"Test F1 Score: {test_results['eval_f1']:.4f}")
print(f"Test Precision: {test_results['eval_precision']:.4f}")
print(f"Test Recall: {test_results['eval_recall']:.4f}")

Test Accuracy: 0.4213
Test F1 Score: 0.4988
Test Precision: 0.6131
Test Recall: 0.4820


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
